### In this notebook we hava created a scraping tool for tripadvisor. This notebooks is an addition to the review scarping tool to gain additional information about restaurants and reviewers.

In [24]:
import pandas as pd
import numpy as np
import requests 
from bs4 import BeautifulSoup
import csv 
from selenium import webdriver
import time
import string
import pickle
import os

In [26]:
# read in the urls needed to scrape
with open('AllRestaurants.txt', "rb") as f:   # Unpickling
    urls = pickle.load(f)

pathtoReviewers = "reviewerInfo.csv"
with open(pathtoReviewers, mode='a', encoding="utf-8") as reviewer_data:
    data_writer = csv.writer(reviewer_data, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
    data_writer.writerow(['username', 'location', 'joined', 'nrContributions','nrReviews', 'nrUpvotes', 'nrFollowers','followers', 'nrFollowing','following'])

pathToStoreInfo = "restaurantInfo.csv"
with open(pathToStoreInfo, mode='a', encoding="utf-8") as trip:
    data_writer = csv.writer(trip, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
    data_writer.writerow(['storeName', 'storeAddress', 'avgRating', 'nrReviews', 'priceCategory','CousineType', 'all_ranks', 'all_ratings'])

In [25]:
def scrapeRestaurantInfo(url):
    page = requests.get(url, timeout=10)
    soup = BeautifulSoup(page.text, 'html.parser')
    storeName = soup.find('h1', class_='_3a1XQ88S').text
    try:
        avgRating = soup.find('span', class_='r2Cf69qf').text.strip()
        nrReviews = soup.find('a', class_='_10Iv7dOs').text.strip().split()[0]
    except:
        avgRating = None
        nrReviews = 0
    storeAddress = soup.find('div', class_= '_2vbD36Hr _36TL14Jn').find('span', class_='_2saB_OSe').text.strip()
#     urlAddress = str(soup.find('div', class_ = '_2vbD36Hr _36TL14Jn').find('span').find('a', href=True)['href'])
    
    try:
        cousineType = [word.text for  word in soup.find('span', class_='_13OzAOXO _34GKdBMV').find_all('a')]
        cousine = True
        if cousineType[0][0] != "$":
            cousineType.insert(0, '')
        
    except:
        cousineType = []
        cousine = False
#     nrPos = soup.find('a', class_='_15QfMZ2L').find('b').find('span').text.strip()
    
    # Other rankings 
    all_ranks = []
    try:
        all_ranks = [word.text for word in soup.find('div', class_ = '_3acGlZjD').find_all('div', class_ = '_3-W4EexF')]
    except:
        all_ranks = []
        
    # Other ratings
    all_ratings = []
    try:
        rating = soup.find_all('div', class_='jT_QMHn2')
        rating_type = [x.find('span', class_ = '_2vS3p6SS').text for x in rating]
        true_rating = [x.find('span', class_ = '_377onWB-') for x in rating]
        true_rating = [int(str(x.findChildren('span')).split('_')[3][:2])/10 for x in true_rating]
        all_ratings = list(zip(rating_type,true_rating))
    except:
        all_ratings = []
    with open(pathToStoreInfo, mode='a', encoding="utf-8") as trip:
        data_writer = csv.writer(trip, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
        if len(cousineType) > 1:
            data_writer.writerow([storeName, storeAddress, avgRating, nrReviews, cousineType[0], cousineType[1:], all_ranks, all_ratings])
        elif len(cousineType) == 1:
            data_writer.writerow([storeName, storeAddress, avgRating, nrReviews, cousineType[0], [], all_ranks, all_ratings])
        else:
            data_writer.writerow([storeName, storeAddress, avgRating, nrReviews, '', [], all_ranks, all_ratings])

In [27]:
with open("AllRestaurants.txt", "rb") as f:   # Unpickling
    urls = pickle.load(f)

bad_url = []
for url in urls:
    try:
        scrapeRestaurantInfo(url)
    except:
        bad_url.append(url)

In [227]:
# GEt all the reviwer info, location, join date, review count, upvotes, followers and following.
def reviewerInfo(url):
    username = url
    full_url = f"https://www.tripadvisor.com/Profile/{url}"
    driver.get(full_url)
    time.sleep(1)

    # Get Intro info, location and join date
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    try:
        location = soup.find('span', class_ = "_2VknwlEe _3J15flPT default").text
    except:
        location = None

    try:
        joined = soup.find('span', class_ = "_1CdMKu4t").text
    except:
        joined = None


    all_links = soup.find_all('div', class_ = '_1aVEDY08')
    # link = driver.find_elements_by_xpath("//div[@class='nkw-3XeH']/div[1]/span[2]/a")

    # # Get the contributions info
    nrContributions = int(str(all_links[0].text).split()[1])
    if nrContributions > 0:
        link = driver.find_elements_by_xpath("//div[@class='nkw-3XeH']/div[1]/span[2]/a")
        driver.execute_script("arguments[0].click();", link[0])
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        nrReviews = int(str(soup.find('span', class_ = 'ui_icon pencil-paper _1LSVmZLi').parent.text).split()[0])
        try:
            nrUpvotes = int(str(soup.find('span', class_ ='ui_icon thumbs-up _1LSVmZLi _3zmXi7gU').parent.text).split()[0])
        except:
            nrUpvotes = 0
        close = driver.find_elements_by_xpath("//div[@class='_2EFRp_bb _9Wi4Mpeb']")
        driver.execute_script("arguments[0].click();", close[0])
    else:
        nrReviews = 0
        nrUpvotes = 0

    # Get Followers
    nrFollowers = int(str(all_links[1].text).split()[1])
    if nrFollowers > 0:
        link = driver.find_elements_by_xpath("//div[@class='nkw-3XeH']/div[2]/span[2]/a")
        driver.execute_script("arguments[0].click();", link[0])
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        followers = [word.text for word in soup.find('div', class_='_1caczhWN').find_all('span', class_='gf69u3Nd')]
        close = driver.find_elements_by_xpath("//div[@class='_2EFRp_bb _9Wi4Mpeb']")
        driver.execute_script("arguments[0].click();", close[0])
    else:
        followers = []

    # Get all following
    nrFollowing = int(str(all_links[2].text).split()[1])
    if nrFollowing > 0:
        link = driver.find_elements_by_xpath("//div[@class='nkw-3XeH']/div[3]/span[2]/a")
        driver.execute_script("arguments[0].click();", link[0])
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        following = [word.text for word in soup.find('div', class_='_1caczhWN').find_all('span', class_='gf69u3Nd')]
        close = driver.find_elements_by_xpath("//div[@class='_2EFRp_bb _9Wi4Mpeb']")
        driver.execute_script("arguments[0].click();", close[0])
    else:
        following = []

    with open(pathtoReviewers, mode='a', encoding="utf-8") as reviewer_data:
        data_writer = csv.writer(reviewer_data, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
        data_writer.writerow([username, location, joined, nrContributions,nrReviews, nrUpvotes, nrFollowers, followers, nrFollowing,following])

In [ ]:
driver_path = f'{os.getcwd()}/chromedriver'
driver = webdriver.Chrome(driver_path)

with open("reviewers.txt", "r") as f:
    reviewers = f.readlines()

for reviewer in reviewers:
    reviewerInfo(reviewer)

In [7]:
urls[0]

'https://www.tripadvisor.com/Restaurant_Review-g189541-d10127436-Reviews-Skagen_Fiskerestaurant_Illum_Rooftop-Copenhagen_Zealand.html'

In [8]:
# Example of how to get the href from googlapis, load the website with selenium and then pull the href attribute.

driver = webdriver.Chrome('/Users/hlynurarnisigurjonsson/Downloads/TripAdvisor-Python-Scraper-Restaurants-2021-main/chromedriver')
url = "https://www.tripadvisor.com/Restaurant_Review-g189541-d694971-Reviews-Noma-Copenhagen_Zealand.html"
driver.get(url)
time.sleep(1)
soup = BeautifulSoup(driver.page_source, 'html.parser')

# more = driver.find_elements_by_xpath("//div[@class='taLnk ulBlueLinks']/a")
# more['href']
# page = requests.get(url)

# soup = BeautifulSoup(page.text, 'html.parser')
soup.find('div', class_ = '_2vbD36Hr _36TL14Jn').find('a').get('href')

'https://maps.google.com/maps?saddr=&daddr=Refshalevej+96%2C+Copenhagen+1432+Denmark@55.68283,12.610479'